#### Imports 

In [1]:
#RNN from the weekly exercises.
from __future__ import absolute_import, division, print_function

#%matplotlib inline
# %matplotlib nbagg
import tensorflow as tf
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
from DataGenerationCommaPlacementWithWordTypes import get_batch_comma
from DataGenerationCommaPlacementWithWordTypes import num_of_training_samples
from DataGenerationCommaPlacementWithWordTypes import get_word_stem_from_encoding

import os
import sys
sys.path.append(os.path.join('.', '..')) 
#from tempUtils import utils

import tf_utils

sentence number 6:  106961
54991 43211 33726 92320 53777 9075 87678 111230 65216 55675 54422 21582 39146 
231 170 142 133 231 118 41 201 223 153 133 238 41 


#### Load previous model
At the bottom of the script there is some code which saves the model.  
If you wish to restore your model from a previous state use this function.

In [25]:
load_model = True 
#model_path = "/tmp/RNN/modelRNNOnlyTypes64.ckpt-530"
model_path = "tmp/modelRNNBig.ckpt-385"

#### Inputdata
Outputs two samples where:   
ENCODED INPUTS is a vector with reference to a word in the dict word_stems.  
ENCODED INPUT TYPES is a vector with reference to the word type in the dict word_types.  
ENCODED TARGETS OUTPUT is the target vector, where 1 is commas and 0 is everything else.  

In [26]:
batch_size = 2
#inputs, inputs_seqlen, targets_in, targets_out, targets_seqlen, targets_mask, \
#text_inputs, text_targets_in, text_targets_out = \
#        get_batch(batch_size=batch_size, max_digits=10, min_digits=5)

inputs, inputs_word_types, inputs_seqlen, targets_in, targets_out, targets_in_seqlen, targets_out_seqlen, targets_mask, \
text_targets_in, text_targets_out = \
        get_batch_comma(batch_size=batch_size, indices_of_interest = [0,1])
    
    
print("input shape:", inputs.shape)
print("input types:", inputs.dtype)#, inputs_seqlen.dtype, targets_in.dtype, targets_out.dtype, targets_seqlen.dtype)

for i in range(batch_size):
    print("\nSAMPLE",i)
    print("ENCODED INPUTS:\t\t\t", inputs[i])
    print("ENCODED INPUT TYPES: \t\t", inputs_word_types[i])
    print("INPUTS SEQUENCE LENGTH:\t\t", inputs_seqlen[i])
    print("TEXT TARGETS OUTPUT:\t\t", text_targets_out[i])
    print("TEXT TARGETS INPUT:\t\t", text_targets_in[i])
    print("ENCODED TARGETS OUTPUT:\t\t", targets_out[i])
    print("ENCODED TARGETS INPUT:\t\t", targets_in[i])
    print("INPUT TARGETS SEQUENCE LENGTH:\t", targets_in_seqlen[i])
    print("OUTPUT TARGETS SEQUENCE LENGTH:\t", targets_out_seqlen[i])
    print("TARGETS MASK:\t\t\t", targets_mask[i])

input shape: (2, 20)
input types: int32

SAMPLE 0
ENCODED INPUTS:			 [ 86374  94286  43211  51222  95495  96390  41472 111230  62764  70687
   4546 115045  71154  80217  98112  53845  12586 115045  97690  39146]
ENCODED INPUT TYPES: 		 [217 217 170 251 268 268 238 201 170 268 238 133 217 153 268 208 217 133
 238  41]
INPUTS SEQUENCE LENGTH:		 20
TEXT TARGETS OUTPUT:		 13#
TEXT TARGETS INPUT:		 #86374 94286 43211 51222 95495 96390 41472 111230 62764 70687 4546 115045 71154 87678 80217 98112 53845 12586 115045 97690 39146 
ENCODED TARGETS OUTPUT:		 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
ENCODED TARGETS INPUT:		 [ 86374  94286  43211  51222  95495  96390  41472 111230  62764  70687
   4546 115045  71154  80217  98112  53845  12586 115045  97690  39146]
INPUT TARGETS SEQUENCE LENGTH:	 20
OUTPUT TARGETS SEQUENCE LENGTH:	 20
TARGETS MASK:			 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]

SAMPLE 1
ENCODED INPUTS:			 [ 83324 116978 117583  82187  91339  

#### Setting up the model

In [39]:
# resetting the graph
tf.reset_default_graph()

# Setting up hyperparameters and general configs
NUM_INPUTS = 133000
NUM_INPUT_TYPES = 317 #a lot of junk is contained there
NUM_OUTPUTS = 2 #0,1,2 #(0-19 +",")

BATCH_SIZE = 256

# try various learning rates 1e-2 to 1e-5
LEARNING_RATE = 0.0005 #0.005
X_EMBEDDINGS = 8
X_TYPES_EMBEDDINGS = 8
t_EMBEDDINGS = 8
NUM_UNITS_ENC = 256 #512
NUM_UNITS_DEC = 256 #512
number_of_layers = 1 #3

# Setting up placeholders, these are the tensors that we "feed" to our network
Xs = tf.placeholder(tf.int32, shape=[None, None], name='X_input')
Xs_types = tf.placeholder(tf.int32, shape=[None, None], name='X_input_types')
ts_in = tf.placeholder(tf.int32, shape=[None, None], name='t_input_in')
ts_out = tf.placeholder(tf.int32, shape=[None, None], name='t_input_out')
X_len = tf.placeholder(tf.int32, shape=[None], name='X_len')
t_in_len = tf.placeholder(tf.int32, shape=[None], name='t_in_len')
t_out_len = tf.placeholder(tf.int32, shape=[None], name='t_out_len')
t_mask = tf.placeholder(tf.float32, shape=[None, None], name='t_mask')
drop = tf.placeholder_with_default(0.0,shape =())

### Building the model
# first we build the embeddings to make our characters into dense, trainable vectors
X_embeddings = tf.get_variable('X_embeddings', [NUM_INPUTS, X_EMBEDDINGS],
                               initializer=tf.random_normal_initializer(stddev=0.1))
X_TYPES_embeddings = tf.get_variable('X_TYPES_embeddings', [NUM_INPUT_TYPES, X_TYPES_EMBEDDINGS],
                               initializer=tf.random_normal_initializer(stddev=0.1))

t_embeddings = tf.get_variable('t_embeddings', [NUM_INPUTS, t_EMBEDDINGS],
                               initializer=tf.random_normal_initializer(stddev=0.1))

X_embedded = tf.gather(X_embeddings, Xs, name='embed_X')
X_TYPES_embedded = tf.gather(X_TYPES_embeddings, Xs_types, name='embed_X_types')
t_embedded = tf.gather(t_embeddings, ts_in, name='embed_t')
input_embedded = tf.concat([X_embedded, X_TYPES_embedded],2)

## forward encoding - use deep cell, with multiple GRU cells stacked on top of each other.

print("input embedded shape:", input_embedded.shape)
def gru_cell():
    return tf.nn.rnn_cell.GRUCell(NUM_UNITS_ENC)
    #return tf.nn.rnn_cell.DropoutWrapper(cell,input_size=8,input_keep_prob=1.0-drop, state_keep_prob=1.0, output_keep_prob=1.0,dtype=tf.float32,
    #                                     variational_recurrent=False)
    #return tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=1.0-drop, dtype=tf.float32, variational_recurrent=True)
stacked_gru = tf.contrib.rnn.MultiRNNCell(
    [gru_cell() for _ in range(number_of_layers)])



enc_out, enc_state = tf.nn.dynamic_rnn(cell=stacked_gru, inputs=input_embedded,
                                  sequence_length=X_len, dtype=tf.float32)
# dynamic Rnn of a stack, outputs the final state of each layer. We are only interested in the final state of the final layer
enc_state = enc_state[-1]

# use below incase TF's makes issues
# enc_state, _ = tf_utils.encoder(X_embedded, X_len, 'encoder', NUM_UNITS_ENC)
#
# enc_state = tf.concat(1, [enc_state, enc_state])

## decoding
# note that we are using a wrapper for decoding here, this wrapper is hardcoded to only use GRU
# check out tf_utils to see how you make your own decoder

# setting up weights for computing the final output
W_out = tf.get_variable('W_out', [NUM_UNITS_DEC, NUM_OUTPUTS])
b_out = tf.get_variable('b_out', [NUM_OUTPUTS])

dec_out, valid_dec_out = tf_utils.decoder(enc_state, t_embedded, tf.shape(t_embedded)[1],
                                          NUM_UNITS_DEC, t_embeddings,
                                          W_out, b_out)

#This code extracts the output of just the last rnn cell in the sequence. Enc_out has the format 
#(Batch_size, max_sequence_length, final state)
#out_tensor = tf.slice(enc_out, [0, 0 ,0],[-1,1,-1])
#valid_out_tensor = tf.slice(enc_out,[0,0,0],[-1,1,-1])

## reshaping to have [batch_size*seqlen, num_units]
out_tensor = tf.reshape(dec_out, [-1, NUM_UNITS_DEC])
valid_out_tensor = tf.reshape(valid_dec_out, [-1, NUM_UNITS_DEC])

# computing output
out_tensor = tf.matmul(out_tensor, W_out) + b_out
valid_out_tensor = tf.matmul(valid_out_tensor, W_out) + b_out

#out_tensor = tf.expand_dims(out_tensor, 1)
#valid_out_tensor = tf.expand_dims(valid_out_tensor, 1)

## reshaping back to sequence
# print('X_len', tf.shape(X_len)[0])
b_size = tf.shape(X_len)[0] # use a variable we know has batch_size in [0]
seq_length = tf.shape(t_embedded)[1] # variable we know has sequence length in [1]
num_out = tf.constant(NUM_OUTPUTS) # casting NUM_OUTPUTS to a tensor variable
out_shape = tf.concat([tf.expand_dims(b_size, 0),
                      tf.expand_dims(seq_length,0),
                      tf.expand_dims(num_out, 0)],
                     axis=0)
outShapeForPrint = tf.Print(out_shape, [out_shape],message= "this is output shape")
out_tensor = tf.reshape(out_tensor, out_shape)

valid_out_tensor = tf.reshape(valid_out_tensor, out_shape)
# handling shape loss
#out_tensor.set_shape([None, None, NUM_OUTPUTS])
y = out_tensor
y_valid = valid_out_tensor

# print all the variable names and shapes
for var in tf.global_variables ():
    s = var.name + " "*(40-len(var.name))
    #print(s, var.value().get_shape())

input embedded shape: (?, ?, 16)


#### Defining loss function, cost functions etc. 

In [40]:
def loss_and_acc(preds):
    # sequence_loss_tensor is a modification of TensorFlow's own sequence_to_sequence_loss
    # TensorFlow's seq2seq loss works with a 2D list instead of a 3D tensors
    loss = tf_utils.sequence_loss_tensor(preds, ts_out, t_mask, NUM_OUTPUTS) # notice that we use ts_out here!

    # if you want regularization
    reg_scale = 0.00001
    regularize = tf.contrib.layers.l2_regularizer(reg_scale)
    params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    reg_term = sum([regularize(param) for param in params])
    loss += reg_term
    
    ## calculate accuracy
    argmax = tf.to_int32(tf.argmax(preds, 2))
    correct = tf.to_float(tf.equal(argmax, ts_out)) * t_mask
    accuracy = tf.reduce_sum(correct) / tf.reduce_sum(t_mask)

## Calculate only relevant part of accuracy
    correct1 = tf.reduce_sum(correct,1)-tf.reduce_sum(t_mask, 1)   
    correct2 = tf.clip_by_value(correct1, -1.0,1.0)
    #correct25 = tf.count_nonzero(correct2, dtype=tf.float32)
    correct3 = tf.subtract(1.0,tf.divide(tf.count_nonzero(correct2, dtype=tf.float32), tf.cast(tf.shape(correct)[0],tf.float32)))
    
    return loss, accuracy, argmax, correct3

loss, accuracy, predictions, cor1 = loss_and_acc(y)
loss_valid, accuracy_valid, predictions_valid, cor1_val = loss_and_acc(y_valid)

# use lobal step to keep track of our iterations
global_step = tf.Variable(0, name='global_step', trainable=False)

# pick optimizer, try momentum or adadelta epsilon has to be higher than the standard, as it will otherwise cause the
#optimizer to go crazy when close to the global optimum.
optimizer = tf.train.AdamOptimizer(LEARNING_RATE, epsilon= 0.001)

# extract gradients for each variable
grads_and_vars = optimizer.compute_gradients(loss)

## add below for clipping by norm
# gradients, variables = zip(*grads_and_vars)  # unzip list of tuples
# clipped_gradients, global_norm = (
#    tf.clip_by_global_norm(gradients, self.clip_norm) )
# grads_and_vars = zip(clipped_gradients, variables)
# apply gradients and make trainable function
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

# print all the variable names and shapes
# notice that we now have the optimizer Adam as well!
for var in tf.global_variables():
    s = var.name + " "*(40-len(var.name))
    #print(s, var.value().get_shape())
print("done")

done


#### Forward pass

In [41]:
## Start the session
# restricting memory usage, TensorFlow is greedy and will use all memory otherwise
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.35)#0.35
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts))#,inter_op_parallelism_threads=4, intra_op_parallelism_threads=4))

# Initialize parameters
if load_model:
    try:
        print(model_path)
        tf.train.Saver().restore(sess, model_path)
    except:
        print("this is the error: ")
        print(sys.exc_info()[1])
        sess.run(tf.global_variables_initializer())
        print('Model not found, new parameters initialized')
else:
    sess.run(tf.global_variables_initializer())

# as always, test the forward pass and initialize the tf.Session!
for i in range(batch_size):
    print("\nSAMPLE",i)
   # print("TEXT INPUTS:\t\t\t", text_inputs[i])
    #print("TEXT TARGETS INPUT:\t\t", text_targets_in[i])

feed_dict = {Xs: inputs, Xs_types: inputs_word_types, X_len: inputs_seqlen, ts_in: targets_in,
             ts_out: targets_out, t_in_len: targets_in_seqlen, t_out_len: targets_out_seqlen}

# test training forwardpass
#print("targets seqlen", targets_in_seqlen)
#print("targets out seqlen", targets_out_seqlen)
fetches = [y, X_embedded, enc_state, input_embedded]
#fetches = [dec_out]
res = sess.run(fetches=fetches, feed_dict=feed_dict)
#print("out shape:", res[0].shape)
#print("inputs", inputs)
#print("input length", inputs_seqlen)
#print("encoder state", res[2].shape)
#print("targets output", targets_out)

print("y", res[0].shape)
print("X_embedded", res[1].shape)
print("input_types_embedded", res[3].shape )

# test validation forwardpass
fetches = [y_valid]
res = sess.run(fetches=fetches, feed_dict=feed_dict)
print("y_valid", res[0].shape)

#Some Data hyperparameters
dropout_perc = 0.7
num_epochs = 2
num_of_training_samples_loaded = num_of_training_samples()
num_of_samples_for_validation = 10000
num_of_training_samples_minus_validation = num_of_training_samples_loaded - num_of_samples_for_validation
num_batches_train = int(num_of_training_samples_minus_validation / BATCH_SIZE)
print("number of batches", num_batches_train)

tmp/modelRNNBig.ckpt-385
INFO:tensorflow:Restoring parameters from tmp/modelRNNBig.ckpt-385

SAMPLE 0

SAMPLE 1
y (2, 20, 2)
X_embedded (2, 20, 8)
input_types_embedded (2, 20, 16)
y_valid (2, 20, 2)
number of batches 378


#### Generate validation data

In [42]:
#Generate some validation data
validation_indices = list(range(num_of_training_samples_minus_validation, num_of_training_samples_minus_validation+num_of_samples_for_validation))
print(len(validation_indices))
X_val, X_types_val, X_len_val, t_in_val, t_out_val, t_len_in_val, t_len_out_val,t_mask_val, \
 text_targets_in_val, text_targets_out_val = \
     get_batch_comma(batch_size=num_of_samples_for_validation, indices_of_interest = validation_indices)
print("X_val", X_val.shape)
print("t_out_val", t_out_val.shape)

10000
X_val (10000, 27)
t_out_val (10000, 27)


#### Training

In [55]:
#%%time
## If you get an error, remove this line! It makes the error message hard to understand.

# setting up running parameters
val_interval = 500
saver = tf.train.Saver()
samples_val = []
epochs, costs, accs_val = [], [], []
plt.figure()
train_acc = []
corr_train = []
accs_val = []
batch_number = 0

if load_model == False:
    try:
        for epoch in range(num_epochs):

            loss_train = 0
            acc_train = 0
            cor_train = 0
            percentage_validation_set_to_evaluate = 2 #20 percent
            print("Epoch: ", epoch)
            if epoch % 5 == 0:
                saver.save(sess,model_path, global_step=epoch)
            for i in range(num_batches_train):
                # load data
                # print("Epoch {0} new Batch: {1} ".format(epoch, i))

                # Select random part of the training data
                idxs = np.random.choice(range(num_of_training_samples_minus_validation), size=(BATCH_SIZE), replace=False)
                X_tr, X_types_tr, X_len_tr, t_in_tr, t_out_tr, t_len_in_tr, t_len_out_tr, t_mask_tr, \
                text_targets_in_tr, text_targets_out_tr = \
                    get_batch_comma(batch_size=BATCH_SIZE, indices_of_interest=idxs)
                # make fetches
                fetches_tr = [train_op, loss, accuracy, cor1]
                # set up feed dict
                feed_dict_tr = {Xs: X_tr, Xs_types: X_types_tr, X_len: X_len_tr, ts_in: t_in_tr,
                                ts_out: t_out_tr, t_in_len: t_len_in_tr, t_out_len: t_len_out_tr, t_mask: t_mask_tr, drop: dropout_perc}
                # run the model
                res = tuple(sess.run(fetches=fetches_tr, feed_dict=feed_dict_tr))
                _, batch_cost, batch_acc, batch_cor = res
                loss_train += batch_cost
                acc_train += batch_acc
                cor_train += batch_cor




            # Validation part:
            print("validating")
            if epoch %10 == 0:
                percentage_validation_set_to_evaluate = 10 #100 percent
            total_acc_val = 0
            cor_val = 0
            for val_part in range(percentage_validation_set_to_evaluate):
                lowerLimit = val_part * 1000
                upperLimit = (val_part+1) *1000
                fetches_val = [accuracy_valid, y_valid, cor1_val]
                feed_dict_val = {Xs: X_val[lowerLimit:upperLimit], Xs_types: X_types_val[lowerLimit:upperLimit],
                                 X_len: X_len_val[lowerLimit:upperLimit], ts_in: t_in_val[lowerLimit:upperLimit],
                                 ts_out: t_out_val[lowerLimit:upperLimit], t_in_len: t_len_in_val[lowerLimit:upperLimit], t_out_len: t_len_out_val[lowerLimit:upperLimit],
                                 t_mask: t_mask_val[lowerLimit:upperLimit]}
                res = tuple(sess.run(fetches=fetches_val, feed_dict=feed_dict_val))
                acc_val, output_val, cor = res
                total_acc_val = total_acc_val+acc_val
                cor_val = cor_val + cor
            total_acc_val = total_acc_val / percentage_validation_set_to_evaluate
            cor_val = cor_val / percentage_validation_set_to_evaluate

            epochs += [epoch]
            accs_val += [cor_val]
            print("Epochs: ", epochs)
            print("accs_val: ", accs_val)
            print("correct_validation_sentences: ", cor_val)
            plt.figure(1)
            plt.plot(epochs, accs_val, 'g-')
            plt.ylabel('Validation Accuracy', fontsize=15)
            plt.xlabel('Epoch', fontsize=15)
            plt.title('', fontsize=20)
            plt.grid('on')
            plt.savefig("out.png")
            display.display(display.Image(filename="out.png"))
            display.clear_output(wait=True)

            #training evaluation
            costs += [loss_train]
            train_acc += [acc_train / num_batches_train]
            corr_train += [cor_train / num_batches_train]

            print("Costs: ", costs)
            print("train Acc:", train_acc)
            print("# of Correct training sentences: ", corr_train)

            plt.figure(1)
            #plt.legend('training Acc')
            plt.ylabel('Acc')
            plt.plot(epochs, corr_train, color="red")
            # plt.plot(updates, KL_valid, color="blue", linestyle="--")
            plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
            plt.grid('on')
            plt.savefig("outTrainAcc.png")

            # plt.subplot(num_classes + 2, 2, 3)
            plt.figure(2)
            plt.legend('training loss')
            plt.ylabel('cost')
            plt.plot(epochs, costs, color="green")
            # plt.plot(updates, KL_valid, color="blue", linestyle="--")
            plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
            plt.grid('on')
            plt.savefig("outLoss.png")

            # plt.subplot(num_classes + 2, 2, 3)

            #evaluate the model
            if epoch % 5 == 0:
                # test the model:
                passed = 0
                failed = 0
                offset = 1000
                numberOfCases = 1000
                # for i in range(1, 10):

                X_tr, X_types_tr, X_len_tr, t_in_tr, t_out_tr, t_len_tr, t_out_len_tr, t_mask_tr, \
                text_targets_in_tr, text_targets_out_tr = \
                    get_batch_comma(batch_size=numberOfCases,
                                    indices_of_interest=validation_indices[offset:numberOfCases + offset])
                # make fetches
                fetches_check = [y, cor1]
                # set up feed dict
                feed_dict_check = {Xs: X_tr, Xs_types: X_types_tr, X_len: X_len_tr, ts_in: t_in_tr,
                                ts_out: t_out_tr, t_in_len: t_len_tr, t_out_len: t_out_len_tr, t_mask: t_mask_tr}
                # run the model
                res = tuple(sess.run(fetches=fetches_check, feed_dict=feed_dict_check))
                for j in range(0, numberOfCases):
                    # print("input", text_inputs_tr)


                    # print("input", text_targets_in_tr)
                    # print("y: ", np.argmax(res[3], axis=2))

                    # convert input back to text:
                    outputStr = ""

                    for k in range(0, X_len_tr[j]):
                        if t_out_tr[j][k] == 1:
                            outputStr += ","

                        outputStr += " "
                        outputStr += get_word_stem_from_encoding(X_tr[j][k])

                    relevant_part_of_facit = np.logical_and(t_out_tr[j], t_mask_tr[j]).astype(int)
                    relevant_part_of_prediction = np.logical_and(np.argmax(res[0][j], axis=1), t_mask_tr[j]).astype(int)
                    np.set_printoptions(suppress=True)
                    if (relevant_part_of_facit - relevant_part_of_prediction).any():
                        print("fail! index", j)
                        print("fail! target: ", t_out_tr[j])
                        print("fail! output: ", 1.0/(1+np.exp(-res[0][j])))
                        print("fail! y argmax: ", np.argmax(res[0][j], axis=1))
                        failed = failed + 1
                        print(outputStr)
                        break
                print("Failed: ", (failed + 0.0) / numberOfCases)
                print("Correct: ", res[1])
    except KeyboardInterrupt:
        pass



print('Done')

Done


#### Plotting

In [59]:
# #plot of validation accuracy for each target position

if load_model == False:
    plt.figure(figsize=(7,7))
    plt.plot(np.mean(np.argmax(output_val,axis=2)==t_out_val,axis=0))
    plt.ylabel('Accuracy', fontsize=15)
    plt.xlabel('Target position', fontsize=15)
    plt.title('', fontsize=20)
    plt.grid('on')
    plt.show()
#why do the plot look like this?

#### Saving the model

In [57]:
# ## Save model
# # Read more about saving and loading models at https://www.tensorflow.org/programmers_guide/saved_model
#
# # Save model
if load_model == False:
    save_path = tf.train.Saver().save(sess, model_path)
    print("Model saved in file: %s" % save_path)

def decode_input_to_string(sequence_length, encoded_input, encoded_input_with_comma):
    # convert input back to text:
    outputStr = ""
    for i in range(0, sequence_length):
        if encoded_input_with_comma[i] == 1:
            outputStr += ","

        outputStr += " "
        outputStr += get_word_stem_from_encoding(encoded_input[i])
    return outputStr

# test the model:
#for i in range (1,10):
passed = 0
failed = 0
countt = 0
offset = 0
numberOfCases = 1000
#for i in validation_indices:
for i in range (1,2):
    X_tr, X_types_tr, X_len_tr, t_in_tr, t_out_tr, t_len_tr, t_out_len_tr, t_mask_tr, \
             text_targets_in_tr, text_targets_out_tr = \
                get_batch_comma(batch_size=numberOfCases,indices_of_interest = validation_indices[offset:numberOfCases+offset])
    # make fetches
    fetches_tr = [train_op, loss, accuracy, y, cor1]
    # set up feed dict
    feed_dict_tr = {Xs: X_tr, Xs_types: X_types_tr, X_len: X_len_tr, ts_in: t_in_tr,
             ts_out: t_out_tr, t_in_len: t_len_tr, t_out_len: t_out_len_tr,t_mask: t_mask_tr}
    # run the model
    res = tuple(sess.run(fetches=fetches_tr, feed_dict=feed_dict_tr))
    #print("input", text_inputs_tr)
    print("result:",res[4])
    for j in range(0,numberOfCases):
        #print("input", text_targets_in_tr[j])
        #print("y: ", np.argmax(res[3][j],axis = 1))

        # convert input back to text:
        output_str = decode_input_to_string(X_len_tr[j], X_tr[j], t_out_tr[j])

        relevant_part_of_facit = np.logical_and(t_out_tr[j], t_mask_tr[j]).astype(int)
        relevant_part_of_prediction = np.logical_and(np.argmax(res[3][j], axis=1), t_mask_tr[j]).astype(int)
        np.set_printoptions(suppress=True)
        #if (relevant_part_of_facit-relevant_part_of_prediction).any():
        print("fail! index", j)
        print("fail! target: ", t_out_tr[j])
        print("y: ", np.argmax(res[3][j], axis=1))
        print("fail! output: ")
        print("", 1.0/(1+np.exp(-res[3][j])))
        print("input", text_targets_in_tr[j])
        print("Sentence:",output_str)
        print("relevant part of prediction ", relevant_part_of_prediction)
        print("")
        failed = failed+1
            #break

print("Failed: ", (failed+0.0)/numberOfCases)



result: 0.927
fail! index 0
fail! target:  [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y:  [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
fail! output: 
 [[ 0.99999976  0.00000028]
 [ 0.97862589  0.02283668]
 [ 0.99999177  0.00000761]
 [ 0.99901414  0.00104397]
 [ 0.75041425  0.27561256]
 [ 0.9696731   0.03645501]
 [ 0.30660439  0.75559705]
 [ 0.99995184  0.00007439]
 [ 0.9999938   0.00001082]
 [ 0.98924118  0.0225765 ]
 [ 0.99981385  0.00039654]
 [ 0.99945968  0.00120751]
 [ 0.99999917  0.00000181]
 [ 0.99993455  0.00014887]
 [ 0.99999428  0.00001231]
 [ 0.99999368  0.00001307]
 [ 0.99999177  0.00001615]
 [ 0.99998832  0.00002165]
 [ 0.99998283  0.00003017]
 [ 0.99997389  0.00004332]
 [ 0.9999584   0.00006626]
 [ 0.99992824  0.00011032]
 [ 0.99987018  0.00019446]
 [ 0.99976629  0.00034419]
 [ 0.99960393  0.00057769]
 [ 0.99938929  0.00088787]]
input #90445 63398 118870 6345 28565 104753 87678 119092 115045 54588 16981 119171 112705 39146 
Sentence:  jacques Lemerciern

input #17138 54422 50383 60313 104048 119473 95290 37005 29766 72662 87678 38534 40829 43894 111524 83929 114353 16981 122225 110749 39146 
Sentence:  Antal af gasværk forblive nogenlunde konstant frem til omkring 1950, da kulgas efterhånden få stærk konkurrence fra anden gastype .
relevant part of prediction  [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

fail! index 33
fail! target:  [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y:  [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
fail! output: 
 [[ 0.99999988  0.00000011]
 [ 0.99999821  0.00000211]
 [ 0.99997509  0.00002201]
 [ 0.99999845  0.00000113]
 [ 0.9999944   0.00000377]
 [ 0.99986875  0.00008363]
 [ 0.99998999  0.00000584]
 [ 0.98675203  0.00808908]
 [ 0.00044941  0.99946827]
 [ 1.          0.00000004]
 [ 0.9999994   0.00000083]
 [ 0.99999332  0.00000989]
 [ 0.99999368  0.00001051]
 [ 0.99999559  0.00000794]
 [ 0.99998593  0.00002889]
 [ 0.99999595  0.00000904]
 [ 0.99999046  0.00002426]
 [ 0.99999988  0


fail! index 119
fail! target:  [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y:  [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
fail! output: 
 [[ 0.99999976  0.0000003 ]
 [ 0.9999795   0.00002227]
 [ 0.99990427  0.00008099]
 [ 0.99996042  0.00002751]
 [ 0.99852955  0.00094827]
 [ 0.99676931  0.00183098]
 [ 0.99997079  0.00001587]
 [ 0.99656725  0.00222408]
 [ 0.99973875  0.00018284]
 [ 0.0000375   0.99996328]
 [ 0.99999893  0.00000134]
 [ 0.99999714  0.00000421]
 [ 0.99997842  0.00004095]
 [ 0.99999011  0.00002519]
 [ 1.          0.00000003]
 [ 0.99999774  0.00000545]
 [ 0.99999142  0.00002206]
 [ 0.99999893  0.00000266]
 [ 0.99999976  0.00000049]
 [ 0.99999392  0.00001518]
 [ 0.99999869  0.00000313]
 [ 0.99999869  0.00000295]
 [ 0.99999881  0.00000236]
 [ 0.99999869  0.00000247]
 [ 0.99999845  0.00000277]
 [ 0.99999797  0.00000342]]
input #130792 131949 51791 29772 115285 107649 23494 111230 116108 87678 128617 23206 45976 26177 125020 61142 43211 19876 101796 39146 


Sentence:  Rejse gå over London til Frankrig og Tyskland, og først 1765 passere man Kbh .
relevant part of prediction  [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

fail! index 206
fail! target:  [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y:  [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
fail! output: 
 [[ 0.99999988  0.00000007]
 [ 0.99999928  0.00000053]
 [ 0.99990308  0.00007884]
 [ 0.90192825  0.08806352]
 [ 0.99944574  0.0005973 ]
 [ 0.9999907   0.00001367]
 [ 0.99967122  0.0005603 ]
 [ 0.00049973  0.99973077]
 [ 0.99999952  0.00000083]
 [ 0.99999988  0.00000023]
 [ 0.99999976  0.00000061]
 [ 0.99999988  0.00000034]
 [ 0.99999988  0.00000037]
 [ 0.99999976  0.00000053]
 [ 0.99999964  0.00000065]
 [ 0.99999952  0.00000072]
 [ 0.99999952  0.0000008 ]
 [ 0.9999994   0.00000097]
 [ 0.99999905  0.00000135]
 [ 0.99999845  0.00000221]
 [ 0.99999702  0.00000406]
 [ 0.99999428  0.00000771]
 [ 0.99998975  0.00001396]
 [ 0.99998355  0.0000228 ]
 [ 0.99997616  0.00

Sentence:  Mangen anden art dyrke i lille målestok, ligeledes for frugt skyld .
relevant part of prediction  [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

fail! index 291
fail! target:  [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
fail! output: 
 [[ 0.99999213  0.00000787]
 [ 0.99993372  0.00006105]
 [ 0.99999273  0.00000567]
 [ 0.99969637  0.00022229]
 [ 0.92772007  0.05080146]
 [ 0.99444699  0.00429478]
 [ 0.65998048  0.30302364]
 [ 0.99990642  0.00009156]
 [ 0.99318975  0.00886044]
 [ 0.9999994   0.00000063]
 [ 0.99814785  0.00260087]
 [ 0.98969346  0.01578508]
 [ 0.97785038  0.03652065]
 [ 0.97808665  0.04108416]
 [ 0.99828988  0.00361815]
 [ 0.04296279  0.98108107]
 [ 0.99970156  0.00077305]
 [ 0.99984109  0.00044523]
 [ 0.99997103  0.00008888]
 [ 0.99999905  0.00000301]
 [ 0.99995315  0.00016066]
 [ 0.999982    0.00005962]
 [ 0.99996877  0.00010023]
 [ 0.99996579  0.00010335]
 [ 0.99997222  0.00007797]
 [

...

input #27968 54422 112297 56195 70887 111230 36597 87678 61776 56798 22305 53845 36632 70293 100236 39146 
Sentence:  Frugt af stjerneanis anvende bl.a. som krydderi, fx i Pernod og den kinesisk køkken .
relevant part of prediction  [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

fail! index 590
fail! target:  [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y:  [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
fail! output: 
 [[ 0.9999994   0.00000061]
 [ 0.99998248  0.00001638]
 [ 0.99999857  0.00000105]
 [ 0.99998951  0.00000646]
 [ 0.99999881  0.00000063]
 [ 0.99984658  0.00009001]
 [ 0.99980241  0.00011884]
 [ 0.99694604  0.00208999]
 [ 0.99618953  0.00359077]
 [ 0.00018598  0.99985588]
 [ 0.99666482  0.00498638]
 [ 0.99995816  0.00007411]
 [ 0.99999535  0.00001022]
 [ 0.99999785  0.00000561]
 [ 0.99998951  0.00002931]
 [ 0.99999774  0.00000635]
 [ 0.99999952  0.00000113]
 [ 0.99998689  0.00003508]
 [ 0.99999726  0.00000695]
 [ 0.99999714  0.00000677]
 [ 0.9999975 

 [[ 0.99999976  0.0000003 ]
 [ 0.99998868  0.00001116]
 [ 0.99998307  0.00001322]
 [ 0.9999001   0.00006884]
 [ 0.99924868  0.00049806]
 [ 0.00037335  0.99962151]
 [ 0.99999988  0.00000012]
 [ 0.99999988  0.00000022]
 [ 0.99999738  0.00000463]
 [ 0.99999511  0.00000954]
 [ 0.9997434   0.00057728]
 [ 0.99999285  0.00001743]
 [ 0.99999356  0.00001745]
 [ 0.99999738  0.00000736]
 [ 0.99999571  0.00001207]
 [ 0.99999321  0.00001978]
 [ 0.99999988  0.00000037]
 [ 0.99999988  0.00000018]
 [ 0.99999666  0.00000857]
 [ 0.99999893  0.00000252]
 [ 0.99999809  0.00000416]
 [ 0.9999975   0.00000485]
 [ 0.99999642  0.00000652]
 [ 0.99999487  0.00000867]
 [ 0.99999225  0.00001222]
 [ 0.99998796  0.00001822]]
input #8437 114847 51222 113395 76560 87678 53845 36632 117583 106604 6096 70887 56798 36632 57829 64977 16981 101241 39146 
Sentence:  innsbruck blive en kulturel centrum, og den stor opsving afspejle bl.a. i den mangen bygning fra periode .
relevant part of prediction  [0 0 0 0 0 1 0 0 0 0 0 0

fail! index 777
fail! target:  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y:  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
fail! output: 
 [[ 0.99999905  0.00000132]
 [ 0.07098172  0.94007409]
 [ 0.99999416  0.00000554]
 [ 0.99999833  0.00000185]
 [ 0.9999994   0.00000079]
 [ 0.99733537  0.00475419]
 [ 1.          0.00000009]
 [ 0.99999082  0.00001782]
 [ 0.99996722  0.00006043]
 [ 0.99999189  0.00001661]
 [ 0.99999917  0.0000017 ]
 [ 0.99997497  0.00005652]
 [ 0.99996889  0.00007212]
 [ 0.99999881  0.00000309]
 [ 0.99999964  0.00000099]
 [ 0.99970335  0.00077223]
 [ 0.99997699  0.00005488]
 [ 0.99996948  0.00006861]
 [ 0.9999727   0.00005751]
 [ 0.99996936  0.00006008]
 [ 0.99996734  0.00005914]
 [ 0.99996293  0.00006237]
 [ 0.99995279  0.00007432]
 [ 0.99993062  0.00010331]
 [ 0.99988329  0.00016672]
 [ 0.99978191  0.00030239]]
input #71444 87678 1129 119034 7537 76479 53845 51222 114933 122913 54422 106796 43211 54261 40105 39146 
Sentence:  Fast, ærbødighed over

 [[ 0.99999774  0.00000184]
 [ 0.99999952  0.00000031]
 [ 0.99996579  0.00001874]
 [ 0.99986482  0.00005976]
 [ 0.99990821  0.0000408 ]
 [ 0.99959141  0.00021255]
 [ 0.9961825   0.00255478]
 [ 0.00025232  0.99973208]
 [ 0.99999082  0.00001077]
 [ 0.99999797  0.00000276]
 [ 0.99999273  0.00001339]
 [ 0.9999944   0.00001189]
 [ 0.99999976  0.00000046]
 [ 0.99999189  0.00002038]
 [ 0.99999785  0.00000562]
 [ 0.99999785  0.0000057 ]
 [ 0.99999785  0.00000556]
 [ 0.99999762  0.0000062 ]
 [ 0.9999975   0.00000597]
 [ 0.99999774  0.00000492]
 [ 0.99999809  0.00000377]
 [ 0.99999833  0.00000299]
 [ 0.99999845  0.00000272]
 [ 0.99999821  0.00000295]
 [ 0.9999975   0.00000385]
 [ 0.99999619  0.00000578]]
input #106730 102486 15292 43211 37005 80682 101102 87678 127997 36632 82848 82913 56195 39146 
Sentence:  Den beskytte virkning være til gengæld god, hvis den rigtig vaccine anvende .
relevant part of prediction  [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

fail! index 870
fail! targe

input #101788 43211 85649 87678 53845 3326 69254 91903 18010 54422 74108 39146 
Sentence:  Oppositionsparti være forbyde, og der ske grov krænkelse af menneskerettighed .
relevant part of prediction  [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

fail! index 975
fail! target:  [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y:  [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
fail! output: 
 [[ 0.99999809  0.0000022 ]
 [ 0.99976903  0.00029954]
 [ 0.99964762  0.00038689]
 [ 0.99887818  0.00121568]
 [ 1.          0.        ]
 [ 0.99999928  0.00000062]
 [ 0.9999162   0.00007856]
 [ 0.99999523  0.00000457]
 [ 0.99214655  0.0090823 ]
 [ 1.          0.00000002]
 [ 0.00008914  0.999933  ]
 [ 0.99985135  0.00017381]
 [ 1.          0.        ]
 [ 0.99999988  0.00000023]
 [ 1.          0.00000009]
 [ 1.          0.00000008]
 [ 0.99999774  0.00000538]
 [ 0.99999964  0.0000007 ]
 [ 0.99999964  0.00000073]
 [ 0.99999964  0.00000067]
 [ 0.99999964  0.00000078]
 [ 0.9999994   0.00

In [24]:

from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file(file_name='tmp/modelRNNBig.ckpt-385', tensor_name='', all_tensors=False)

W_out (DT_FLOAT) [256,2]
W_out/Adam (DT_FLOAT) [256,2]
W_out/Adam_1 (DT_FLOAT) [256,2]
X_TYPES_embeddings (DT_FLOAT) [317,8]
X_TYPES_embeddings/Adam (DT_FLOAT) [317,8]
X_TYPES_embeddings/Adam_1 (DT_FLOAT) [317,8]
X_embeddings (DT_FLOAT) [133000,8]
X_embeddings/Adam (DT_FLOAT) [133000,8]
X_embeddings/Adam_1 (DT_FLOAT) [133000,8]
b_out (DT_FLOAT) [2]
b_out/Adam (DT_FLOAT) [2]
b_out/Adam_1 (DT_FLOAT) [2]
beta1_power (DT_FLOAT) []
beta2_power (DT_FLOAT) []
decoder/W_c_h (DT_FLOAT) [256,256]
decoder/W_c_h/Adam (DT_FLOAT) [256,256]
decoder/W_c_h/Adam_1 (DT_FLOAT) [256,256]
decoder/W_c_x (DT_FLOAT) [8,256]
decoder/W_c_x/Adam (DT_FLOAT) [8,256]
decoder/W_c_x/Adam_1 (DT_FLOAT) [8,256]
decoder/W_r_h (DT_FLOAT) [256,256]
decoder/W_r_h/Adam (DT_FLOAT) [256,256]
decoder/W_r_h/Adam_1 (DT_FLOAT) [256,256]
decoder/W_r_x (DT_FLOAT) [8,256]
decoder/W_r_x/Adam (DT_FLOAT) [8,256]
decoder/W_r_x/Adam_1 (DT_FLOAT) [8,256]
decoder/W_z_h (DT_FLOAT) [256,256]
decoder/W_z_h/Adam (DT_FLOAT) [256,256]
decoder/W_z_

#### Close and save

In [39]:
sess.close()